In [3]:
!pip install -q --upgrade nltk gensim bokeh pandas

import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 1.5 MB 5.3 MB/s 
     |████████████████████████████████| 23.9 MB 92 kB/s 
     |████████████████████████████████| 11.5 MB 43.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.1 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

## Запилим пословный машинный перевод!

In [6]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

Archive:  cc.ru.300.vec.zip
  inflating: cc.ru.300.vec           
Archive:  cc.uk.300.vec.zip
  inflating: cc.uk.300.vec           


Напишем простенькую реализацию модели машинного перевода.

Идея основана на статье [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). У авторов в репозитории еще много интересного: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

А мы будем переводить с украинского на русский.

![](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)   
*синій кіт* vs. *синій кит*

In [7]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

Посмотрим на пару серпень-август (являющихся переводом)

In [8]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [9]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [10]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

In [11]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

In [168]:
uk_ru_train

[('iснує', 'существует'),
 ('абияк', 'как-нибудь'),
 ('або', 'или'),
 ('або', 'ли'),
 ('абсолютно', 'совершенно'),
 ('абсолютно', 'совсем'),
 ('автомобіль', 'автомобиль'),
 ('автомобіль', 'вагон'),
 ('агов', 'эй'),
 ('аґрус', 'крыжовник'),
 ('адже', 'ведь'),
 ('адміністрація', 'администрация'),
 ('адреса', 'адрес'),
 ('академік', 'академик'),
 ('академія', 'академия'),
 ('актор', 'артист'),
 ('акуратно', 'аккуратно'),
 ('акції', 'действие'),
 ('акція', 'акция'),
 ('але', 'же'),
 ('але', 'зато'),
 ('але', 'но'),
 ('американець', 'американец'),
 ('аналіз', 'анализ'),
 ('арешт', 'арест'),
 ('аркушів', 'листы'),
 ('армія', 'войско'),
 ('атрибут', 'признак'),
 ('багато', 'гораздо'),
 ('багато', 'многие'),
 ('багато', 'много'),
 ('багато', 'многое'),
 ('базовий', 'база'),
 ('бак', 'танк'),
 ('батальйон', 'батальон'),
 ('батьки', 'родители'),
 ('батьків', 'отцы'),
 ('батьківщина', 'родина'),
 ('батько', 'отец'),
 ('бачити', 'видать'),
 ('бачити', 'видеть'),
 ('безглуздий', 'бессмысленный'),
 

In [13]:
X_test[[1]]

array([[ 7.960e-02,  9.800e-03,  1.550e-02,  2.400e-03,  1.500e-03,
        -2.240e-02,  1.040e-02,  2.590e-02, -3.150e-02,  2.900e-02,
         2.300e-03, -1.400e-02,  8.700e-03, -9.570e-02, -1.670e-02,
        -2.090e-02,  1.440e-02,  2.200e-03,  2.780e-02,  4.320e-02,
         3.400e-02, -5.290e-02, -2.490e-02, -3.370e-02, -1.140e-02,
         1.290e-02, -8.780e-02, -5.400e-02, -1.610e-02,  7.210e-02,
        -3.200e-03, -5.100e-03,  3.750e-02,  6.000e-03, -4.000e-03,
         2.830e-02,  3.600e-02,  3.180e-02,  3.880e-02,  6.250e-02,
         4.870e-02,  8.800e-03,  4.940e-02, -9.800e-03, -4.540e-02,
        -2.700e-03, -6.910e-02, -1.180e-02,  8.300e-03,  5.800e-03,
         5.500e-03, -2.880e-02,  4.590e-02,  2.410e-02, -2.720e-02,
         7.170e-02, -2.900e-03, -4.100e-03,  1.810e-02, -1.490e-02,
        -1.180e-02,  1.800e-02,  4.100e-03,  2.270e-02,  4.300e-03,
        -9.090e-02, -2.540e-02, -6.000e-04,  4.510e-02, -1.970e-02,
         4.260e-02, -4.090e-02, -2.210e-02,  1.8

In [14]:
Y_train

array([[-0.0138, -0.0218, -0.0034, ..., -0.0052,  0.0157, -0.0186],
       [-0.0124, -0.0632,  0.0116, ...,  0.0112,  0.058 , -0.0469],
       [ 0.0069, -0.1661,  0.0196, ..., -0.0125, -0.0673, -0.0716],
       ...,
       [-0.1443,  0.0515, -0.088 , ..., -0.0526,  0.0573, -0.0543],
       [ 0.0455, -0.0164, -0.1064, ...,  0.0484,  0.0396, -0.0998],
       [ 0.0059, -0.0262,  0.0177, ...,  0.0155,  0.043 , -0.0433]],
      dtype=float32)

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [15]:
from sklearn.linear_model import LinearRegression
mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Проверим, куда перейдет `серпень`:

In [16]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923630714417),
 ('июль', 0.8234528303146362),
 ('август', 0.8120501637458801),
 ('декабрь', 0.8039003610610962)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [17]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    ru_vectors = [ru[1] for ru in pairs]
    assert len(pairs) == len(ru_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs, 0):
        ru_vec = ru_emb.most_similar(mapped_vectors[[i]])[:topn]
        pred_words = [ru[0] for ru in ru_vec]
        if ru_vectors[i] in pred_words:
            num_matches += 1

    precision_val = num_matches / len(pairs)
    return precision_val

In [18]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [19]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [20]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [68]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    P, D, Q = np.linalg.svd(np.matmul(X_train.T, Y_train), full_matrices=False)

    return np.dot(P, Q)

In [69]:
W = learn_transform(X_train, Y_train)

(300, 300) (300,) (300, 300)


In [65]:
X_train.shape

(1840, 300)

In [66]:
W.shape

(1840, 1840)

In [70]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [71]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

0.6537467700258398


### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [74]:
with open("fairy_tale.txt", "r") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [78]:
for s in uk_sentences:
    print(s)
    print()

лисичка - сестричка і вовк - панібрат

як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !

дай мені огня " .

а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .

прибігла на поле , аж там пасуть хлопці бичків .

вона і каже їм : " ей , хлопці !

проміняйте мені бичка - третячка за маковий пирожок " .

тії согласились ; так вона їм говорить : " смотріть же , ви не їжте зараз сього пирожка , а тоді уже розломите , як я заведу бичка за могилку ; а то ви його ні за що не розломите " .

бачите вже - лисичка таки собі була розумна , що хоть кого да обманить .

тії хлопці так і зробили , а лисичка як зайшла за могилу , да зараз у ліс і повернула , щоб на дорозі не догнали ; пр

In [167]:
ukrain_words = [x[0] for x in uk_ru_test]
ukrain_words_2 = [x[0] for x in uk_ru_train]
ukrain_words = ukrain_words + ukrain_words_2
if 'w' in ukrain_words:
    print('мишу')

ukrain_words

мишу


['або',
 'активний',
 'актор',
 'але',
 'асамблея',
 'бабуся',
 'багажник',
 'бажати',
 'башта',
 'бізнес',
 'бійка',
 'блискавка',
 'братися',
 'будиночок',
 'будівництво',
 'важливо',
 'валити',
 'вантаж',
 'вдячність',
 'величність',
 'вечеря',
 'вечірній',
 'вибирати',
 'визнати',
 'використати',
 'винести',
 'вирок',
 'виступити',
 'відвести',
 'відкриватися',
 'відмінний',
 'відрізати',
 'вік',
 'вісім',
 'вітати',
 'вовк',
 'водити',
 'водій',
 'володіти',
 'впевненість',
 'врахувати',
 'врятувати',
 'втручатися',
 'вухо',
 'вхід',
 'газети',
 'галузь',
 'галявина',
 'геній',
 'говорити',
 'голодний',
 'готель',
 'готовність',
 'громадянин',
 'грудень',
 'ґрунт',
 'дах',
 'двісті',
 'двічі',
 'де',
 'деревини',
 'дзеркало',
 'дивитися',
 'дивитися',
 'дивізія',
 'дивуватися',
 'діаманти',
 'дід',
 'днів',
 'довести',
 'довести',
 'доводити',
 'дозволити',
 'до',
 'допустити',
 'дотепи',
 'дотепно',
 'дурний',
 'духовний',
 'експедиція',
 'електричний',
 'елемент',
 'етап',
 'жал

In [187]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    splitted_list = sentence.split(' ')
    bag_of_words = []
    for word in splitted_list:
        try:
            ru_vec = ru_emb.most_similar([np.matmul(uk_emb[word], W)])[:1]
            pred_words = [ru[0] for ru in ru_vec]
            bag_of_words.append(pred_words)
        except KeyError:
            bag_of_words.append([word])
    return ' '.join(str(e) for e in flatten(bag_of_words))

In [183]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [188]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к однії бабы добувать огня ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи пирожок со маком , сладкий , че и согнула , чтобы он прохолов ; а лисичка ой и підгледала , че токмо что бабка качнулась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок че и деру со хаты , че , пробежать , весь мак со его виїла , а туда мусо

# Дополнительные материалы

## Почитать
### База:  
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[Deep Learning, NLP, and Representations, Christopher Olah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)  

### Как кластеризовать смыслы многозначных слов:  
[Making Sense of Word Embeddings (2016), Pelevina et al](http://anthology.aclweb.org/W16-1620)    

### Как оценивать эмбеддинги
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel](http://www.aclweb.org/anthology/D15-1036)  
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu](https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)  
[Problems With Evaluation of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui](https://arxiv.org/pdf/1605.02276.pdf)  
[Improving Reliability of Word Similarity Evaluation by Redesigning Annotation Task and Performance Measure (2016), Oded Avraham, Yoav Goldberg](https://arxiv.org/pdf/1611.03641.pdf)  
[Evaluating Word Embeddings Using a Representative Suite of Practical Tasks (2016), N. Nayak](https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)  


## Посмотреть
[Word Vector Representations: word2vec, Lecture 2, cs224n](https://www.youtube.com/watch?v=ERibwqs9p38)